# Generalized Grover's Algorithm in CudaQ

In [1]:
import numpy as np
import cudaq

## CudaQ Docs Grover's Example

If anyone can figure out how this is supposed to run, be my guest.

In [ ]:
# @cudaq.kernel
# def reflect(qubits: cudaq.qview):
#     ctrls = qubits.front(qubits.size() - 1)
#     last = qubits.back()
#     cudaq.compute_action(lambda: (h(qubits), x(qubits)),
#                           lambda: z.ctrl(ctrls, last))

# @cudaq.kernel
# def oracle(q: cudaq.qview):
#     z.ctrl(q[0], q[2])
#     z.ctrl(q[1], q[2])

# @cudaq.kernel
# def grover(N: int, M: int, oracle: Callable[[cudaq.qview], None]):
#     q = cudaq.qvector(N)
#     h(q)
#     for i in range(M):
#         oracle(q)
#         reflect(q)
#     mz(q)

# counts = cudaq.sample(grover, 3, 1, oracle)
# assert len(counts) == 2
# assert '101' in counts
# assert '011' in counts

## Generalized Grover's

### Multi-Control Toffoli Gate

In [7]:
@cudaq.kernel
def kernel():
    qvector = cudaq.qvector(5)
    x(qvector)
    x.ctrl(qvector[0:3], qvector[4])
    mz(qvector)


results = cudaq.sample(kernel)
print(results)

{ 11110:1000 }



In [56]:
DEBUG = True

@cudaq.kernel
def init_qbits(reg: cudaq.qview, val: 'int'):
    # Convert decimal input to binary, clear 0b prefix
    for index, bit in enumerate(reg):
        if index == val:
            x(bit)



# @cudaq.kernel
# def oracle(register: cudaq.qview, auxillary_qubit: cudaq.qubit,
#            hidden_bits: List[int]):
#     for index, bit in enumerate(hidden_bits):
#         if bit == 1:
#             # apply a `cx` gate with the current qubit as
#             # the control and the auxillary qubit as the target.
#             x.ctrl(register[index], auxillary_qubit)

In [57]:
@cudaq.kernel
def kernel():
    # Allocate the specified number of qubits - this
    # corresponds to the length of the hidden bitstring.
    qubits = cudaq.qvector(5)
    init_qbits(qubits, 3)

    # Apply measurement gates to just the `qubits`
    # (excludes the auxillary qubit).
    mz(qubits)


# print(cudaq.draw(kernel))
result = cudaq.sample(kernel)
print(result)

UnboundLocalError: local variable 'id' referenced before assignment

In [ ]:
import cudaq
import random

from typing import List


def random_bits(length: int):
    bitset = []
    for _ in range(length):
        bitset.append(random.randint(0, 1))
    return bitset


# If you have a NVIDIA GPU you can use this example to see
# that the GPU-accelerated backends can easily handle a
# larger number of qubits compared the CPU-only backend.

# Depending on the available memory on your GPU, you can
# set the number of qubits to around 30 qubits, and un-comment
# the `cudaq.set_target(nvidia)` line.

# Note: Without setting the target to the `nvidia` backend,
# there will be a noticeable decrease in simulation performance.
# This is because the CPU-only backend has difficulty handling
# 30+ qubit simulations.

qubit_count = 5  # set to around 30 qubits for `nvidia` target
# ```
# cudaq.set_target("nvidia")
# ```

# Generate a random, hidden bitstring for the oracle
# to encode. Note: we define the bitstring here so
# as to be able to return it for verification.
hidden_bits = random_bits(qubit_count)


@cudaq.kernel
def oracle(register: cudaq.qview, auxillary_qubit: cudaq.qubit,
           hidden_bits: List[int]):
    for index, bit in enumerate(hidden_bits):
        if bit == 1:
            # apply a `cx` gate with the current qubit as
            # the control and the auxillary qubit as the target.
            x.ctrl(register[index], auxillary_qubit)


@cudaq.kernel
def bernstein_vazirani(hidden_bits: List[int]):
    # Allocate the specified number of qubits - this
    # corresponds to the length of the hidden bitstring.
    qubits = cudaq.qvector(len(hidden_bits))
    # Allocate an extra auxillary qubit.
    auxillary_qubit = cudaq.qubit()

    # Prepare the auxillary qubit.
    h(auxillary_qubit)
    z(auxillary_qubit)

    # Place the rest of the register in a superposition state.
    h(qubits)

    # Query the oracle.
    oracle(qubits, auxillary_qubit, hidden_bits)

    # Apply another set of Hadamards to the register.
    h(qubits)

    # Apply measurement gates to just the `qubits`
    # (excludes the auxillary qubit).
    mz(qubits)


print(cudaq.draw(bernstein_vazirani, hidden_bits))
result = cudaq.sample(bernstein_vazirani, hidden_bits)

print(f"encoded bitstring = {hidden_bits}")
print(f"measured state = {result.most_probable()}")
print(
    f"Were we successful? {''.join([str(i) for i in hidden_bits]) == result.most_probable()}"
)